In [81]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
import datetime

# setup the tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


@tool
def square(a: int) -> int:
    """Calculates the square of a number."""
    a = int(a)
    return a * a

@tool
def give_increment(a: int) -> float:
    """if the number is negative then make it postive number and then return the number * 1.25"""
    result = abs(a) * 1.25
    return result

@tool
def give_customise_increment(a: int, b: int) -> float:
    """if the number is negative then make it postive number and return the number * b/100"""
    result = abs(a) * b/100
    return result

@tool
def add_salution_to_name(name: str) -> str:
    """Modify then name with which starts with capital letter and prefix a salution before the name such as Mr. or Mrs.
    According to gender of the name, it will add Mr. or Mrs. before the name.
    If you unable to identify the name then return the name with Prefix 'Hello Dear'
    """

@tool
def friend_name_passed_from_human_input(name: str) -> str:
    """Extract the name (multiple name if exists) from the input then, prefix the name with salution before the name such as Mr. or Mrs.
    According to gender of the name, it will add Mr. or Mrs. before the name.
    If you are unable to identify the name then return the name with Prefix 'Hello Dear'
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a mathematical assistant.
        Use your tools to answer questions. If you do not have a tool to
        answer the question, say so. 

        Return answer with description in a new line, e.g
        Human: What is 1 + 1?
        AI: Invoked Tool tool_name and the result is : 1 + 1 = 2
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Choose the LLM that will drive the agent
llm = ChatGroq(model="llama3-8b-8192", temperature=0.5)

# setup the toolkit
toolkit = [add, multiply, square, give_increment, give_customise_increment, add_salution_to_name]

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)
print(datetime.datetime.now())

2024-10-09 15:15:46.679258


In [ ]:
print("-----------------")
result = agent_executor.invoke({"input": "what is 1 + 1?"})
print(result['output'])
print("+++++++++++++++++")

print("-----------------")
result = agent_executor.invoke({"input": "how much to give on increment on -300 and 150"})
print(result['output'])
print("+++++++++++++++++")

print("-----------------")
result = agent_executor.invoke({"input": "how much to give on increment on 1000 with 10%"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...


In [78]:
print("-----------------")
result = agent_executor.invoke({"input": "my name is krishna bhunia and I need to address me properly"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j9btedz7fsz9geae1j5afeh3` on : Limit 500000, Used 500618, Requested 100. Please try again in 2m4.1008s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [72]:
print("-----------------")
result = agent_executor.invoke({"input": "My friend name is priyanka and abhijit"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add_salution_to_name` with `{'name': 'Priyanka'}`


None
Invoking: `add_salution_to_name` with `{'name': 'Abhijit'}`


NoneIt seems that the tools did not yield any results. Let me try to provide a direct response.

Mr. Priyanka and Mr. Abhijit

> Finished chain.
It seems that the tools did not yield any results. Let me try to provide a direct response.

Mr. Priyanka and Mr. Abhijit
+++++++++++++++++


In [73]:
print("-----------------")
result = agent_executor.invoke({"input": "My friend name is Alex"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add_salution_to_name` with `{'name': 'Alex'}`


None
Invoking: `add_salution_to_name` with `{'name': 'Alex'}`
responded: I apologize for the inconvenience. It seems that the tool call "call_bgs9" did not yield a valid result. Let me try again.



NoneHello Dear Alex

> Finished chain.
Hello Dear Alex
+++++++++++++++++


In [74]:
print("-----------------")
result = agent_executor.invoke({"input": "My friend name is Alexa"})
print(result['output'])
print("+++++++++++++++++")

-----------------


> Entering new AgentExecutor chain...

Invoking: `add_salution_to_name` with `{'name': 'Alexa'}`


None
Invoking: `add_salution_to_name` with `{'name': 'Alexa'}`
responded: It seems that the tool call did not yield a result. Let me try again.



NoneHello Dear Alexa

> Finished chain.
Hello Dear Alexa
+++++++++++++++++
